In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from lxml import etree

In [2]:
url = 'https://remaxaruba.com/property/list'

In [3]:
def get_all_url(li):
    all_links = []
    req = requests.get(url)
    soup = BeautifulSoup(req.content, "html.parser")
    anchors = soup.find('div', {'class': 'page-listing'})
    links  = anchors.find_all('a', href = True)
    for link in links:
        urls = link.get('href')
        #print(urls)
        all_links.append(urls)
    return all_links
path = get_all_url(url)

In [4]:
path2 = []
 
for i in path:
    if i not in path2:
        path2.append(i)
path2

['https://remaxaruba.com/property/details/Modern-Two-Story-Complex',
 'https://remaxaruba.com/property/details/coral-shell-loft',
 'https://remaxaruba.com/property/details/av-milio-croes-46g',
 'https://remaxaruba.com/property/details/la-colina-25-2',
 'https://remaxaruba.com/property/details/Medical-Offices-For-Sale',
 'https://remaxaruba.com/property/details/klip-straat-20-ladies-choice',
 'https://remaxaruba.com/property/details/across-from-the-airport',
 'https://remaxaruba.com/property/details/Opal%20133%20Beautiful%20&%20Spacious',
 'https://remaxaruba.com/property/details/esmeralda-121-c',
 'https://remaxaruba.com/property/details/tuscany-residence-luxury-villa-14',
 'https://remaxaruba.com/property/details/esmeralda-125',
 'https://remaxaruba.com/property/details/bubali-villa-park-113-l',
 'https://remaxaruba.com/property/details/divi-the-residences-penthouse-42',
 'https://remaxaruba.com/property/details/palm-beach-514',
 'https://remaxaruba.com/property/details/wilhelminastra

In [5]:
def get_details(urls):
    property_list = []
    for link in urls:
        req = requests.get(link)
        soup = BeautifulSoup(req.content, 'html.parser')
        dom = etree.HTML(str(soup))
        name = dom.xpath('//*[@id="layout-content"]/div[2]/div/div/div[1]/h1')[0].text
        location = dom.xpath('//*[@id="layout-content"]/div[3]/div/div/div[1]/div/div[2]/div/div/div[1]/ul/li[2]/div/div[2]')[0].text.strip()
        try:
            price = soup.find('div', class_ = "price").text.strip().split(' ')[1]
        except AttributeError:
            price = None
        try:
            rooms = dom.xpath('//*[@id="layout-content"]/div[3]/div/div[2]/div[1]/div/div[2]/div/div/div[1]/ul/li[3]/div/div[2]')[0].text.strip()
            bathroom = dom.xpath('//*[@id="layout-content"]/div[3]/div/div[2]/div[1]/div/div[2]/div/div/div[1]/ul/li[4]/div/div[2]')[0].text.strip()
            pool = dom.xpath('//*[@id="layout-content"]/div[3]/div/div[2]/div[1]/div/div[2]/div/div/div[2]/ul/li[5]/div/div[2]')[0].text.strip()
            property_status = dom.xpath('//*[@id="layout-content"]/div[3]/div/div[1]/span[1]')[0].text
        except IndexError:
            rooms = dom.xpath('//*[@id="layout-content"]/div[3]/div/div/div[1]/div/div[2]/div/div/div[1]/ul/li[3]/div/div[2]')[0].text.strip()
            bathroom = dom.xpath('//*[@id="layout-content"]/div[3]/div/div/div[1]/div/div[2]/div/div/div[1]/ul/li[4]/div/div[2]')[0].text.strip()
            pool = dom.xpath('//*[@id="layout-content"]/div[3]/div/div/div[1]/div/div[2]/div/div/div[2]/ul/li[5]/div/div[2]')[0].text.strip()
            property_status = None
        lat = dom.xpath('//*[@id="map"]/script[1]')[0].text.strip().split('{')[2].split(',')[0].split(':')[1].strip()
        long = dom.xpath('//*[@id="map"]/script[1]')[0].text.strip().split('{')[2].split(',')[1].split(':')[1].strip().replace('}', "")
            
        prop = {
            'name': name,
            'location': location,
            'property': property_status,
            'price': price,
            'bedrooms': rooms,
            'bathrooms' : bathroom,
            'Pool':pool,
            'Latitude': lat,
            'Longitude':long,
            'link': link
        }
        property_list.append(prop)
        print('Saving:', prop['name'])
        
    df = pd.DataFrame(property_list)
    print(df.head(20))
    df.to_csv('remaxaruba.csv', header=True, index=False)

In [8]:
get_details(path2)

Saving: Modern Commercial Building Complex
Saving: Coral Shell Condominium Loft Unit 6
Saving: Oranjestad Luxury Villa
Saving: Tierra del Sol - La Colina 25
Saving: Commercial Offices near Eagle
Saving: Klip straat # 20, Ladies Choice
Saving: Land near Airport
Saving: Opal Luxury Villa 133
Saving: Esmeralda 121 C
Saving: Tuscany Luxury Villa 14
Saving: Esmeralda 125
Saving: Bubali Villa Park 113-L
Saving: Divi The Residences Penthouse 42
Saving: Palm Beach 514
Saving: Wilhelminastraat 102/104
Saving: Atlantic 360
Saving: Frere Alexius
Saving: Coral Shell Condominium Unit 2
Saving: Pekinini 11
Saving: Gold Coast Diamante Luxury Villa 129
Saving: Coral Shell Condominium Unit 13
Saving: Scopetstraat 12
Saving: Tanki Leendert Commercial Building
Saving: Piedra Plat 110-A
Saving: The Daher Residence *** Only 3 Unites Left***
Saving: The Daher Residence - Suite 3
Saving: Bubali 109 Apartment Complex
Saving: Gold Coast Luxury Villa Diamante 176
Saving: Bakval Villa 7J
Saving: Kamay Villa
Savi